In [45]:
from pprint import pprint
from typing import List
import mlflow
import ujson as json
from mlflow.tracking import MlflowClient
import sys
print(sys.path)
sys.path.append("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline")


['/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/src/clean', '/home/alextay96/anaconda3/envs/work/lib/python310.zip', '/home/alextay96/anaconda3/envs/work/lib/python3.10', '/home/alextay96/anaconda3/envs/work/lib/python3.10/lib-dynload', '', '/home/alextay96/.local/lib/python3.10/site-packages', '/home/alextay96/anaconda3/envs/work/lib/python3.10/site-packages', '/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline']


In [46]:

from data import ImportEnv
import pathlib
import os
from tqdm import tqdm
import glob
import pandas as pd
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import DataLoader2
import torch
from torch.cuda.amp import autocast
import torchmetrics
import numpy as np
import itertools
import ast
from joblib import Parallel, delayed
from collections import Counter
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    recall_score,
    confusion_matrix,
)
from collections import Counter
import awswrangler as wr
from src.TrainClassifierParams import trainParams
import warnings
from cleanlab.filter import find_label_issues
from cleanlab.internal.multilabel_utils import onehot2int
from cleanlab.outlier import OutOfDistribution

from cleanlab.dataset import health_summary
from cleanlab.multilabel_classification import get_label_quality_scores

import torchvision
import matplotlib.pyplot as plt
import cv2
import plotly.express as px
from skimage import io


In [47]:


def get_view_names():
    return [
        "front_view_left",
        "front_view",
        "front_view_right",
        "rear_view",
        "rear_view_left",
        "rear_view_right",
    ]


def get_cv_pred(expId, vehicleType, downloadDir, view):
    outputDir = pathlib.Path(downloadDir)
    os.makedirs(outputDir, exist_ok=True)
    runName = f"cv_pred_{vehicleType}_{view}"
    query = f"tags.`mlflow.runName`='{runName}'"
    runs = MlflowClient().search_runs(
        experiment_ids=[expId],
        filter_string=query,
        order_by=["attribute.start_time DESC"],
        max_results=1,
    )
    info = runs[0].info
    runId = info.run_id
    mlflow.artifacts.download_artifacts(run_id=runId, dst_path=outputDir)


def combine_df(downloadDir, view):
    search = f"{downloadDir}/cv_pred_**{view}.csv"
    allPredfile = glob.glob(search, recursive=False)
    allDf = []
    print(allPredfile)
    for p in allPredfile:
        viewName = p.split("/")[-1].split(".")[0].replace("cv_pred_", "")
        print(viewName)
        df = pd.read_csv(p)
        df["view"] = viewName
        allDf.append(df)
    completeDf = pd.concat(allDf)
    completeDf.dropna(subset="file", inplace=True)
    return completeDf


def compile_pred_proba(completeDf: pd.DataFrame, allParts: List[str], valuesCol: str):
    completeDf["CaseID"] = completeDf["file"].apply(
        lambda x: int(x.split("/")[-1].split("_")[0])
    )
    df_wide = completeDf.pivot_table(index="CaseID", columns="part", values=valuesCol)
    caseFileDf = completeDf.groupby("CaseID").head(1)[["CaseID", "file", "view"]]
    assert len(caseFileDf) == len(df_wide)
    filesDf = df_wide.merge(caseFileDf[["file", "view", "CaseID"]], on="CaseID")
    df_wide.sort_values(by="CaseID", inplace=True)
    filesDf.sort_values(by="CaseID", inplace=True)
    # print(completeDf[["file", "view", "CaseID"]])
    return df_wide, filesDf


In [48]:
downloadDir = "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/cleanlab"
expId = 108
vehicleType = "Saloon - 4 Dr"
views = get_view_names()
view = views[0] 
get_cv_pred(expId, vehicleType, downloadDir, view)
completePredDf = combine_df(downloadDir, view)
allParts = completePredDf["part"].unique().tolist()
allParts = [x for x in allParts if x not in ["vision_misc", "vision_non_external"]]
viewPredDf, filesDf = compile_pred_proba(completePredDf, allParts, "conf")
viewGtDf, filesDf = compile_pred_proba(completePredDf, allParts, "gt")
labels = onehot2int(viewGtDf.values)
pred_labels = onehot2int(np.array((viewPredDf.values > 0.5), np.int32))
ranked_label_issues = find_label_issues(
    labels=labels,
    pred_probs=viewPredDf.values,
    multi_label=True,
    return_indices_ranked_by="self_confidence",
    n_jobs=1,
    filter_by="confident_learning",
)


['/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/cleanlab/cv_pred_Saloon - 4 Dr_front_view_left.csv']
Saloon - 4 Dr_front_view_left


In [49]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)


In [50]:
image_names = np.array([i for i in filesDf["file"].tolist()])  # name of file for each image
image_names

array(['/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/10000038_438742006.JPG',
       '/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/10000084_437534248.JPG',
       '/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/10000104_437290603.JPG',
       ...,
       '/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/13074208_595299174.JPG',
       '/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/13075282_595114804.JPG',
       '/home/alextay96/Desktop/new_workspace/DLDataPipeline/data/imgs/13076951_595070513.JPG'],
      dtype='<U85')

In [51]:
class_names = viewPredDf.columns
def get_label_name(ls):
    return [class_names[i] for i in ls]
label_names = [get_label_name(i) for i in labels]
pred_label_names = [get_label_name(i) for i in pred_labels]


In [53]:
ranked_label_issues

array([5810, 6144, 8616, ..., 7079, 1860, 7358])

In [54]:
def plot_image(pos, pred_label=None):
    """ Plots particular CelebA image and its label, and optionally the model-predicted label as well. """
    label_error = label_names[pos]
    path = image_names[pos]
    # print(image_names[pos].split("/")[-1])
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = io.imread(path)
    plt.axis("off")
    plt.imshow(img)
    caseId = int( image_names[pos].split("/")[-1].split("_")[0])
    caseMetadata = multilabelDf[multilabelDf["CaseID"] == caseId]
    # print(caseMetadata[["CaseID", "Circumstances_of_Accident", "Claim_Type", "Model", "Collision_With"]])
    caseDataDf = caseMetadata[["CaseID", "Circumstances_of_Accident", "Claim_Type", "Model", "Collision_With"]]
    # fig = px.imshow(img)
    # fig.show()
    fontsize = 20
    plt.text(
        800,
        0,
        image_names[pos].split("/")[-1],
        bbox=dict(
            fill=False,
            edgecolor="black",
            linewidth=1,
        ),
        fontsize=fontsize,
    )
    txt = plt.text(
        800,
        400,
        "Given Label: " + str(label_error),
        bbox=dict(fill=False, edgecolor="red", linewidth=1),
        fontsize=fontsize,
        wrap=True,
    )
    txt._get_wrap_line_width = lambda: 400
    if pred_label is not None:
        txt = plt.text(
            1500,
            400,
            "Model Predicted: " + str(pred_label),
            bbox=dict(fill=False, edgecolor="green", linewidth=1),
            fontsize=fontsize,
            wrap=True,
        )
        txt._get_wrap_line_width = lambda: 400
    cell_text = []
    for row in range(len(caseDataDf)):
        cell_text.append(caseDataDf.iloc[row])

    t2 = plt.table(cellText=cell_text, colLabels=caseDataDf.columns)
    plt.axis('off')
    t2.scale(5, 5)
    t2.set_fontsize(30)

In [55]:
scores = get_label_quality_scores(labels, viewPredDf.values)
# scores[:10]

In [56]:
scores.shape

(10000,)

In [57]:
def display_issues(ct):
    rows = 5

    fig = plt.figure(figsize=(8, 40))
    columns = 1
    for i in range(1, columns * rows + 1):
        pos = ranked_label_issues[ct]
        label_error = label_names[pos]
        
        fig.add_subplot(
            rows,
            columns,
            i,
        )
        plot_image(pos, pred_label_names[pos])
        ct += 1

In [58]:
def get_label_df(filename):
    labelDf = wr.s3.read_csv(path=f"s3://imgs_labels/{filename}")

    return labelDf

In [259]:
targetImgLabelFilename = f"{vehicleType}_{view}_img_labels.csv"
currentVerImgLabelDf = get_label_df(targetImgLabelFilename)
currentVerImgLabelDf

,Unnamed: 0,vision_bonnet,vision_bumper_front,vision_door_front_lh,vision_door_rear_lh,vision_engine,vision_fender_front_lh,vision_front_panel,vision_headlamp_lh,vision_non_external,vision_misc,vision_wheel,vision_windscreen_front,CaseID,view,filename
0,0,1,1,0,0,0,0,0,0,0,0,0,0,12686836,front_view_left,12686836_575779946.JPG
1,1,1,1,0,0,1,1,1,1,1,1,1,1,12687681,front_view_left,12687681_575529975.JPG
2,2,1,1,0,0,1,0,1,1,0,0,0,0,12691867,front_view_left,12691867_575749697.JPG
3,3,1,1,0,0,1,0,1,1,0,0,0,0,12876912,front_view_left,12876912_585741948.JPG
4,4,1,0,0,0,0,0,0,0,0,0,0,0,10608489,front_view_left,10608489_467975674.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,0,0,0,0,0,0,0,0,0,0,10061030,front_view_left,10061030_440392598.JPG
9996,9996,0,0,0,0,0,0,0,0,0,0,0,0,10102008,front_view_left,10102008_442386066.JPG
9997,9997,0,0,0,0,0,0,0,0,0,0,0,0,10138474,front_view_left,10138474_444192656.JPG
9998,9998,0,0,0,0,0,0,0,0,0,0,0,0,10143783,front_view_left,10143783_444927378.JPG


In [260]:
ranked_label_issues

array([5810, 6144, 8616, ..., 7079, 1860, 7358])

In [261]:
topHalfLabelIssue = ranked_label_issues[:len(ranked_label_issues) // 2]

In [262]:
allParts

['vision_bonnet',
 'vision_bumper_front',
 'vision_door_front_lh',
 'vision_door_rear_lh',
 'vision_engine',
 'vision_fender_front_lh',
 'vision_front_panel',
 'vision_headlamp_lh',
 'vision_wheel',
 'vision_windscreen_front']

In [263]:
targetPartToCorrect = ["vision_bonnet", "vision_bumper_front", 
"vision_windscreen_front", 
"vision_engine",
"vision_grille", "vision_bumper_rear", "vision_rear_compartment",  'vision_wheel']

In [264]:
# currentAvailablePartToCorrect = sorted(list(set(targetPartToCorrect) & set(allParts)))
currentAvailablePartToCorrect = sorted(allParts)
currentAvailablePartToCorrect

['vision_bonnet',
 'vision_bumper_front',
 'vision_door_front_lh',
 'vision_door_rear_lh',
 'vision_engine',
 'vision_fender_front_lh',
 'vision_front_panel',
 'vision_headlamp_lh',
 'vision_wheel',
 'vision_windscreen_front']

In [265]:
allCaseWithLabelIssue = viewGtDf.index.values[topHalfLabelIssue]
allCaseWithLabelIssue

array([10368095, 10405785, 10990537, ..., 10597085, 10203385, 12915435])

In [266]:
from loguru import logger

In [267]:
correctedLabelDf = currentVerImgLabelDf.copy(deep=True)
for targetPart in currentAvailablePartToCorrect:
    fpCaseToConvertDf = correctedLabelDf[
                        (correctedLabelDf["CaseID"].isin(allCaseWithLabelIssue)) &
                        (correctedLabelDf[targetPart] == 1)
                        ][[targetPart, "CaseID"]]
    posWithoutIssue = len(correctedLabelDf[(~correctedLabelDf["CaseID"].isin(fpCaseToConvertDf["CaseID"])) & 
                                    ( (correctedLabelDf[targetPart] == 1))])
    noIssuePosLabelRatio =  posWithoutIssue / len(correctedLabelDf)
    if(noIssuePosLabelRatio < 0.1):
        logger.warning(f'Skipping {targetPart} due low pos label after filtering')
        continue
    logger.success(f"{targetPart} : {noIssuePosLabelRatio} pos label left with no issues")
    '''
    Convert labels to zero
    '''
    correctedLabelDf.loc[correctedLabelDf["CaseID"].isin(fpCaseToConvertDf["CaseID"].unique().tolist()), targetPart] = 0


2022-12-30 17:47:31.719 | SUCCESS  | __main__:<module>:13 - vision_bonnet : 0.2136 pos label left with no issues
2022-12-30 17:47:31.722 | SUCCESS  | __main__:<module>:13 - vision_bumper_front : 0.4149 pos label left with no issues
2022-12-30 17:47:31.724 | WARNING  | __main__:<module>:11 - Skipping vision_door_front_lh due low pos label after filtering
2022-12-30 17:47:31.725 | WARNING  | __main__:<module>:11 - Skipping vision_door_rear_lh due low pos label after filtering
2022-12-30 17:47:31.727 | SUCCESS  | __main__:<module>:13 - vision_engine : 0.2894 pos label left with no issues
2022-12-30 17:47:31.729 | SUCCESS  | __main__:<module>:13 - vision_fender_front_lh : 0.121 pos label left with no issues
2022-12-30 17:47:31.731 | SUCCESS  | __main__:<module>:13 - vision_front_panel : 0.1623 pos label left with no issues
2022-12-30 17:47:31.734 | SUCCESS  | __main__:<module>:13 - vision_headlamp_lh : 0.2773 pos label left with no issues
2022-12-30 17:47:31.736 | WARNING  | __main__:<modu

In [268]:
correctedLabelDf

,Unnamed: 0,vision_bonnet,vision_bumper_front,vision_door_front_lh,vision_door_rear_lh,vision_engine,vision_fender_front_lh,vision_front_panel,vision_headlamp_lh,vision_non_external,vision_misc,vision_wheel,vision_windscreen_front,CaseID,view,filename
0,0,1,1,0,0,0,0,0,0,0,0,0,0,12686836,front_view_left,12686836_575779946.JPG
1,1,1,1,0,0,1,1,1,1,1,1,1,1,12687681,front_view_left,12687681_575529975.JPG
2,2,1,1,0,0,1,0,1,1,0,0,0,0,12691867,front_view_left,12691867_575749697.JPG
3,3,0,0,0,0,0,0,0,0,0,0,0,0,12876912,front_view_left,12876912_585741948.JPG
4,4,1,0,0,0,0,0,0,0,0,0,0,0,10608489,front_view_left,10608489_467975674.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,0,0,0,0,0,0,0,0,0,0,10061030,front_view_left,10061030_440392598.JPG
9996,9996,0,0,0,0,0,0,0,0,0,0,0,0,10102008,front_view_left,10102008_442386066.JPG
9997,9997,0,0,0,0,0,0,0,0,0,0,0,0,10138474,front_view_left,10138474_444192656.JPG
9998,9998,0,0,0,0,0,0,0,0,0,0,0,0,10143783,front_view_left,10143783_444927378.JPG


In [269]:
correctedLabelDf.loc[correctedLabelDf["CaseID"].isin(fpCaseToConvertDf["CaseID"].unique().tolist()), "vision_headlamp_lh"]

21      0
32      0
51      0
62      0
69      0
       ..
9509    0
9510    0
9678    0
9707    0
9716    0
Name: vision_headlamp_lh, Length: 539, dtype: int64

In [270]:

viewGtDf.index.values[5810]

10368095

In [271]:
viewGtDf.loc[10368095]

part
vision_bonnet              1.0
vision_bumper_front        0.0
vision_door_front_lh       0.0
vision_door_rear_lh        0.0
vision_engine              1.0
vision_fender_front_lh     0.0
vision_front_panel         1.0
vision_headlamp_lh         0.0
vision_misc                1.0
vision_non_external        1.0
vision_wheel               0.0
vision_windscreen_front    1.0
Name: 10368095, dtype: float64

In [272]:
# display_issues(0)

In [273]:
# display_issues(5)

In [274]:
# display_issues(10)

In [275]:
# display_issues(15)

In [276]:
# display_issues(20)

In [277]:
# display_issues(25)

In [278]:
# display_issues(30)

In [279]:
# display_issues(35)

In [280]:
# display_issues(40)

In [281]:
# display_issues(45)

In [282]:
# display_issues(50)